 O objetivo é desenvolver um modelo preditivo capaz de antecipar qual será a Nota de Avaliação (Review Score) de um pedido antes mesmo que o cliente a escreva.

## IMPORTANDO DADOS E BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)             # permite a visualização de 50 colunas do dataframe
#pd.options.display.float_format = '{:.2f}'.format    # pandas: para todos os números aparecerem com duas casas decimais

In [2]:
customers = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_customers_dataset.csv")
sellers = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_sellers_dataset.csv")
order_items = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_order_items_dataset.csv")
products = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_products_dataset.csv")
geolocation = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_geolocation_dataset.csv")
category_name_translation = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/product_category_name_translation.csv")
order_payments = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_order_payments_dataset.csv")
pedidos_treinamento = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/TREINAMENTO/pedidos_treinamento.csv")
reviews_treinamento = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/TREINAMENTO/reviews_treinamento.csv")
pedidos_teste = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/TESTE/pedidos_teste.csv")

In [3]:
print(f"customers: {customers.shape}")
print(f"sellers: {sellers.shape}")
print(f"order_items: {order_items.shape}")
print(f"products: {products.shape}")
print(f"geolocation: {geolocation.shape}")
print(f"category_name_translation: {category_name_translation.shape}")
print(f"order_payments: {order_payments.shape}")
print(f"pedidos_treinamento: {pedidos_treinamento.shape}")
print(f"reviews_treinamento: {reviews_treinamento.shape}")
print(f"pedidos_teste: {pedidos_teste.shape}")

customers: (99441, 5)
sellers: (3095, 4)
order_items: (112650, 7)
products: (32951, 9)
geolocation: (1000163, 5)
category_name_translation: (71, 2)
order_payments: (103886, 5)
pedidos_treinamento: (74887, 8)
reviews_treinamento: (75385, 7)
pedidos_teste: (20943, 8)


## Tabela pedidos_treinamento (principal)

### Anotações

In [4]:
# Tamanho da Tabela
pedidos_treinamento.shape

(74887, 8)

In [5]:
# Tipos dos atributos
pedidos_treinamento.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [6]:
# Número de valores únicos por atributo
pedidos_treinamento.nunique()

order_id                         74887
customer_id                      74887
order_status                         2
order_purchase_timestamp         74488
order_approved_at                69611
order_delivered_carrier_date     67803
order_delivered_customer_date    74249
order_estimated_delivery_date      375
dtype: int64

In [7]:
# Verificando se há valores nulos
pedidos_treinamento.isna().sum()

order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      1
order_delivered_customer_date     0
order_estimated_delivery_date     0
dtype: int64

In [8]:
# Verificando se há valores duplicados
pedidos_treinamento.loc[pedidos_treinamento.duplicated()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [9]:
# Visualização dos dados
pedidos_treinamento.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
2,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
3,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01 00:00:00
4,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07 00:00:00


## Tabela reviews_treinamento (principal)

### Anotações

In [10]:
# Tamanho da Tabela
reviews_treinamento.shape

(75385, 7)

In [11]:
# Tipos dos atributos
reviews_treinamento.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [12]:
# Número de valores únicos por atributo
reviews_treinamento.nunique()

review_id                  74721
order_id                   74887
review_score                   5
review_comment_title        1595
review_comment_message     27416
review_creation_date         546
review_answer_timestamp    74603
dtype: int64

In [13]:
# Verificando se há valores nulos
reviews_treinamento.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       72039
review_comment_message     44425
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [14]:
# Verificando se há valores duplicados
reviews_treinamento.loc[reviews_treinamento.duplicated()]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp


In [15]:
# Visualização dos dados
reviews_treinamento.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


## Tabela customers

### Anotações

In [16]:
# Tamanho da Tabela
customers.shape

(99441, 5)

In [17]:
# Tipos dos atributos
customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [18]:
# Número de valores únicos por atributo
customers.nunique()

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

In [19]:
# Verificando se há valores nulos
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [20]:
# Verificando se há valores duplicados
customers.loc[customers.duplicated()]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [21]:
# Visualização dos dados
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## Tabela sellers 

### Anotações

In [22]:
# Tamanho da Tabela
sellers.shape

(3095, 4)

In [23]:
# Tipos dos atributos
sellers.dtypes

seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object

In [24]:
# Número de valores únicos por atributo
sellers.nunique()

seller_id                 3095
seller_zip_code_prefix    2246
seller_city                611
seller_state                23
dtype: int64

In [25]:
# Verificando se há valores nulos
sellers.isna().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [26]:
# Verificando se há valores duplicados
sellers.loc[sellers.duplicated()]

,seller_id,seller_zip_code_prefix,seller_city,seller_state


In [27]:
# Visualização dos dados
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## Tabela order_items

### Anotações

In [28]:
# Tamanho da Tabela
order_items.shape

(112650, 7)

In [29]:
# Tipos dos atributos
order_items.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [30]:
# Número de valores únicos por atributo
order_items.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

In [31]:
# Verificando se há valores nulos
order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [32]:
# Verificando se há valores duplicados
order_items.loc[order_items.duplicated()]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


In [33]:
# Visualização dos dados
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


## Tabela products

### Anotações

In [34]:
# Tamanho da Tabela
products.shape

(32951, 9)

In [35]:
# Tipos dos atributos
products.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [36]:
# Número de valores únicos por atributo
products.nunique()

product_id                    32951
product_category_name            73
product_name_lenght              66
product_description_lenght     2960
product_photos_qty               19
product_weight_g               2204
product_length_cm                99
product_height_cm               102
product_width_cm                 95
dtype: int64

In [37]:
# Verificando se há valores nulos
products.isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [38]:
# Verificando se há valores duplicados
products.loc[products.duplicated()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [39]:
# Visualização dos dados
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


## Tabela geolocation

### Anotações

In [40]:
# Tamanho da Tabela
geolocation.shape

(1000163, 5)

In [41]:
# Tipos dos atributos
geolocation.dtypes

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [42]:
# Número de valores únicos por atributo
geolocation.nunique()

geolocation_zip_code_prefix     19015
geolocation_lat                717360
geolocation_lng                717613
geolocation_city                 8011
geolocation_state                  27
dtype: int64

In [43]:
# Verificando se há valores nulos
geolocation.isna().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

In [44]:
# Verificando se há valores duplicados
geolocation.loc[geolocation.duplicated()]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
15,1046,-23.546081,-46.644820,sao paulo,SP
44,1046,-23.546081,-46.644820,sao paulo,SP
65,1046,-23.546081,-46.644820,sao paulo,SP
66,1009,-23.546935,-46.636588,sao paulo,SP
67,1046,-23.546081,-46.644820,sao paulo,SP
...,...,...,...,...,...
1000153,99970,-28.343273,-51.873734,ciriaco,RS
1000154,99950,-28.070493,-52.011342,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS


In [45]:
# Visualização dos dados
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


## Tabela category_name_translation

### Anotações
- This one is prety useless

In [46]:
# Visualização dos dados
category_name_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [47]:
# Visualização dos dados
category_name_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


## Tabela order_payments

### Anotações

In [48]:
# Tamanho da Tabela
order_payments.shape

(103886, 5)

In [49]:
# Tipos dos atributos
order_payments.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [50]:
# Número de valores únicos por atributo
order_payments.nunique()

order_id                99440
payment_sequential         29
payment_type                5
payment_installments       24
payment_value           29077
dtype: int64

In [51]:
# Verificando se há valores nulos
order_payments.isna().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [52]:
# Verificando se há valores duplicados
order_payments.loc[order_payments.duplicated()]

,order_id,payment_sequential,payment_type,payment_installments,payment_value


In [53]:
# Visualização dos dados
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


## Tabela pedidos_teste

### Anotações

In [54]:
# Tamanho da Tabela
pedidos_teste.shape

(20943, 8)

In [55]:
# Tipos dos atributos
pedidos_teste.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [56]:
# Número de valores únicos por atributo
pedidos_teste.nunique()

order_id                         20943
customer_id                      20943
order_status                         1
order_purchase_timestamp         20843
order_approved_at                18324
order_delivered_carrier_date     12220
order_delivered_customer_date    20773
order_estimated_delivery_date      126
dtype: int64

In [57]:
# Verificando se há valores nulos
pedidos_teste.isna().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

In [58]:
# Verificando se há valores duplicados
pedidos_teste.loc[pedidos_teste.duplicated()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [59]:
# Visualização dos dados
pedidos_teste.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
1,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
2,82566a660a982b15fb86e904c8d32918,d3e3b74c766bc6214e0c830b17ee2341,delivered,2018-06-07 10:06:19,2018-06-09 03:13:12,2018-06-11 13:29:00,2018-06-19 12:05:52,2018-07-18 00:00:00
3,5ff96c15d0b717ac6ad1f3d77225a350,19402a48fe860416adf93348aba37740,delivered,2018-07-25 17:44:10,2018-07-25 17:55:14,2018-07-26 13:16:00,2018-07-30 15:52:25,2018-08-08 00:00:00
4,dcb36b511fcac050b97cd5c05de84dc3,3b6828a50ffe546942b7a473d70ac0fc,delivered,2018-06-07 19:03:12,2018-06-12 23:31:02,2018-06-11 14:54:00,2018-06-21 15:34:32,2018-07-04 00:00:00
